In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
import astropy as ap
import matplotlib.pyplot as plt
import scipy.stats
import astropy.time
import astropy.units as u
import astropy.constants as c

# style = "https://raw.githubusercontent.com/fedhere/DSPS/master/fbb.mplstyle"
# plt.style.use(style)

In [ ]:
def mul_err(x, y, dx=0, dy=0):
    """
    Multiply two numbers with errors and propagate them.
    """
    z = x * y
    dz = np.abs(z) * np.sqrt((dx/x)**2 + (dy/y)**2)
    return z, dz


def div_err(x, y, dx=0, dy=0):
    """
    Divide two numbers with errors and propagate them.
    """
    z = x / y
    dz = np.abs(z) * np.sqrt((dx/x)**2 + (dy/y)**2)
    return z, dz

In [ ]:
m31_csv = \
"https://raw.githubusercontent.com/fedhere/DSPS_FBianco/master/HW4/m31.csv"
mw_csv = \
"https://raw.githubusercontent.com/fedhere/DSPS_FBianco/master/HW4/mw.csv"

In [ ]:
m31 = pd.read_csv(m31_csv, sep=",", skiprows=1)
mw = pd.read_csv(mw_csv, sep=",", skiprows=1)

In [ ]:
new_colnames = ["r", "v", "dv", "err"]

m31_colnames = list(m31.columns)
m31_new_col_dict = {colname: new_colname for colname, new_colname 
                    in zip(m31_colnames, new_colnames)}

mw_colnames = list(mw.columns)
mw_new_col_dict = {colname: new_colname for colname, new_colname 
                   in zip(mw_colnames, new_colnames)}

In [ ]:
m31 = m31.rename(m31_new_col_dict, axis=1)
mw = mw.rename(mw_new_col_dict, axis=1)

In [ ]:
def physical_size(d, theta):
    """
    Return the physical size of an object from its angular size and distance.
    """
    L = 2 * d * np.tan(theta / 2)
    return L

In [ ]:
m31_angular_size = (3.167 * u.deg).to(u.rad)
m31_distance = (2.54 * 10**6 * u.lyr).to(u.kpc)
m31_distance_err = (0.11 * 10**6 * u.lyr).to(u.kpc)
m31_physical_size = physical_size(m31_distance, m31_angular_size)

In [ ]:
print(f"Angular size of luminous Andromeda: {m31_angular_size:.5f}")
print(f"Physical size of luminous Andromeda: {m31_physical_size:.1f}")

In [ ]:
def velocity_at_nr(gal, gal_size, n):
    """
    Finds the last row in the data table where r <= gal_size * n.
    """
    data = m31[m31["r"] <= gal_size * n].iloc[-1]
    return data

In [ ]:
velocity_at_nr(m31, m31_physical_size.value, 100)

In [ ]:
def plot_GRC(ax, gal, gal_size=None, n=None, c=None, label=None, text=False):
    
    if n is not None:
        data = velocity_at_nr(gal, gal_size, n)
    else:
        data = gal
        
    if c is None:
        c = "tab:blue"

    ax.errorbar(data["r"], data["v"],
                yerr=data["dv"], c=c,
                capsize=4, linewidth=2,
                alpha=0.5)

    ax.errorbar(data["r"], data["v"],
                yerr=data["err"], c=c,
                capsize=4, linewidth=2,
                label=label)
    
    if text:
        annotation = f"{data['v']:.1f}\n({data['dv']:.1f}, {data['err']:.1f})"
        ax.text(data["r"], data["v"] - data["dv"], annotation,
                ha="center", va="top")
        

def plotGal(gal, gal_size):
    fig, ax = plt.subplots(figsize=(12, 6))
    # ax.set_xscale("log")
    plot_GRC(ax, gal, label="GRC")

    plot_GRC(ax, gal,
             gal_size=gal_size, n=1,
             c="tab:orange", label="GRC at Gal Radius", text=True)
    plot_GRC(ax, gal,
             gal_size=gal_size, n=2,
             c="tab:green", label="GRC at 2x Gal Radius", text=True)
    plot_GRC(ax, gal,
             gal_size=gal_size, n=3,
             c="tab:red", label="GRC at 3x Gal Radius", text=True)

    plt.xlabel("Radius [kpc]")
    plt.ylabel("Velocity [km / s]")
    
    
    plt.xlim((0.1, 80))
    plt.legend(fontsize=15)
    plt.show()

In [ ]:
plotGal(m31, m31_physical_size.value / 2)

### Figure 1: The galactic rotation curve for M31 (Andromeda) is shown out to a radius of 80 kpc, just over three times the radius of the luminous mass in M31. Long and short error bars are displayed, representing modified standard deviations and modified errors. When all mass is included, we expect the velocity to decreas as 1/r, however this is not what is observed. This result is interpreted as the galaxy having more matter that is not luminous at large radii.

In [ ]:
def GRC_summary(gal, gal_size, n, err="dv"):
    data0 = velocity_at_nr(gal, gal_size, 1)
    dataf = velocity_at_nr(gal, gal_size, n)
    
    ratio, stderr = div_err(dataf["r"], data0["r"],
                            dx=dataf[err], dy=data0[err])
    
    print(f"{ratio:.3f} ± {stderr:.3f}")
    

GRC_summary(m31, m31_physical_size.value, 3, err="err")

In [ ]:
int_r = np.arange(1, 6)
int_r